In [5]:
Folder1 = "test/txt"
Folder2 = "train/beth/txt"
Folder3 = "train/partners/txt"

In [144]:
import os
import re
import time
import requests

In [44]:
# Functions -start-

In [130]:
BlackList = [
    "YES", "NO", "NORMAL", "REPORT", "OTHER", "SIZE", "TIME", "DATE", "DOCTOR", "TIME", "PLAN", "FOLLOW", "MEDICINE", "REVIEW", "COUNT", 
]

In [131]:
def GetAllFilesListInFolder(f):
    files = os.listdir(f)
    textFiles = []
    
    for file in files:
        if file.endswith(".txt"):
            textFiles.append(os.path.join(f, file))
    return textFiles


def ReadAllFilesInList(Files):
    content = []
    for file in Files:
        with open(file, "r", encoding='utf-8-sig') as f:
            content.append(f.read())
    return content

def FindAbbreviations(text):
    # (?=[A-Z0-9]*[A-Z])
    # İfadenin devamında sıfır veya daha fazla "büyük harf ve rakam" ve ardından en az bir büyük harf olmalıdır.
    #
    # [A-Z0-9]{2,}
    # En az bir büyük harf ve rakamdan oluşan, en az 2 karakterlik veri
    #
    # Kombinasyonlu ifade de bu ifadelerin arasına "/" karakteri gelirse de kabul ediyor.
    # DM2, PCRE, DM2/A (kombinasyonlu)
    abbreviation_pattern = r'\b(?=[A-Z0-9]*[A-Z])[A-Z0-9]{2,6}\b'
    combined_pattern = r'\b(?=[A-Z0-9]*[A-Z])[A-Z0-9]{2,6}/(?=[A-Z0-9]*[A-Z])[A-Z0-9]{1,5}\b'

    tokens = re.findall(abbreviation_pattern, text)
    tokens.extend(re.findall(combined_pattern, text))

    abbreviations = {token for token in tokens if re.search(r'[A-Z]', token) and token not in BlackList and not token[0].isdigit()}

    return sorted(abbreviations)

def FilterAbbreviationsWithTreshold(allAbbreviations, threshold):
    # Çok fazla kısaltma bulunurken, bunların bazıları yanlış olabilir.
    # Az sayıda tekrar eden veri varsa bunları sileceğiz, belirli bir eşik değerin üstündeki
    # tekrar sayıları o verinin potansiyel kısaltma olduğunu ifade edeceğini kabul ediyoruz.
    abbreviationsCount = {}

    for abbreviation in allAbbreviations:
        abbreviationsCount[abbreviation] = abbreviationsCount.get(abbreviation, 0) + 1

    return [abbr for abbr, count in abbreviationsCount.items() if count > threshold]

In [132]:
# Functions -end-

In [133]:
Files1 = GetAllFilesListInFolder(Folder1)
Files2 = GetAllFilesListInFolder(Folder2)
Files3 = GetAllFilesListInFolder(Folder3)

Content1 = ReadAllFilesInList(Files1)
Content2 = ReadAllFilesInList(Files2)
Content3 = ReadAllFilesInList(Files3)

In [134]:
print(str(len(Files1)) + " + " + str(len(Files2)) + " + " + str(len(Files3)) + " = " + str(len(Files1) + len(Files2) + len(Files3)) + " Files" )

256 + 73 + 97 = 426 Files


In [138]:
AllAbbreviations = []

for Contents in [Content1, Content2, Content3]:
    for Content in Contents:
        arr = FindAbbreviations(Content)
        AllAbbreviations.extend(arr)

AllAbbreviations = FilterAbbreviationsWithTreshold(AllAbbreviations, 3)
AllAbbreviations = set(AllAbbreviations)
print(AllAbbreviations)

{'A1', 'TEE', 'CELL', 'YEAST', 'BP', 'DR', 'BM', 'CD', 'LIPASE', 'CH', 'GGI', 'MRA', 'POS', 'GERD', 'RWH', 'C1', 'ETT', 'MARK', 'SGPT', 'SKIN', 'LYMPHS', 'NEURO', 'NP', 'V2', 'XII', 'PAIN', 'US', 'MRN', 'S2', 'COR', 'ACUTE', 'ATRIAL', 'RISS', 'RIGHT', 'CC', 'RESUME', 'ABD', 'MUMPS', 'ABG', 'VQ', 'ALT', 'RCA', 'KETONE', 'GH', 'AT', 'CMV', 'S4', 'ERCP', 'TOT', 'REASON', 'Q8H', 'BXH', 'SQ', 'NODES', 'ORDERS', 'PO', 'APPEAR', 'PARKS', 'CENTER', 'PPI', 'TUE', 'NON', 'RATE', 'Q6H', 'PAST', 'FULL', 'MASS', 'RETURN', 'CBC', 'TR', 'QAM', 'AR', 'CK', 'MED', 'CA', 'HLGMC', 'MM', 'IN', 'EOMI', 'SVR', 'BID', 'OBS', 'WORK', 'NVH', 'LDH', 'SEEN', 'SM', 'EGD', 'AND', 'VULVA', 'SODIUM', 'STAT', 'IVC', 'IF', 'JVD', 'UTI', 'PTT', 'ENT', 'WEEKS', 'NONE', 'DP', 'RPR', 'AV', 'SITE', 'NIACIN', 'WOUND', 'D50', 'LD', 'LDL', 'LV', 'INH', 'XRT', 'PNA', 'HCTZ', 'DIET', 'CTMC', 'PPD', 'MD', 'MEALS', 'PCP', 'BRIEF', 'CHEST', 'MCHC', 'OF', 'SSS', 'FRI', 'OR', 'HOURS', 'CRI', 'LUNG', 'PHD', 'CNS', 'HEIGHT', 'TURP', '

In [ ]:
# Terimler sözlüğü uygulanacak

In [120]:
URL = "https://clinicaltables.nlm.nih.gov/api/conditions/v3/search?terms=[WORD]&limit=1"

In [121]:
def GetURL(word):
    return URL.replace("[WORD]", word)

In [146]:
CorrectTerms = []

it = 0
for i in AllAbbreviations:
    print(i)
    it = it + 1
    url = GetURL(i)
    
    response = requests.get(url)
    if response.text != "[0,[],null,[]]":
        CorrectTerms.append(i)
    else:
        url = url.replace("conditions", "procedures")
        response = requests.get(url)
        if response.text != "[0,[],null,[]]":
            CorrectTerms.append(i)

A1
TEE
CELL
YEAST
BP
DR
BM
CD
LIPASE
CH
GGI
MRA
POS
GERD
RWH
C1
ETT
MARK
SGPT
SKIN
LYMPHS
NEURO
NP
V2
XII
PAIN
US
MRN
S2
COR
ACUTE
ATRIAL
RISS
RIGHT
CC
RESUME
ABD
MUMPS
ABG
VQ
ALT
RCA
KETONE
GH
AT
CMV
S4
ERCP
TOT
REASON
Q8H
BXH
SQ
NODES
ORDERS
PO
APPEAR
PARKS
CENTER
PPI
TUE
NON
RATE
Q6H
PAST
FULL
MASS
RETURN
CBC
TR
QAM
AR
CK
MED
CA
HLGMC
MM
IN
EOMI
SVR
BID
OBS
WORK
NVH
LDH
SEEN
SM
EGD
AND
VULVA
SODIUM
STAT
IVC
IF
JVD
UTI
PTT
ENT
WEEKS
NONE
DP
RPR
AV
SITE
NIACIN
WOUND
D50
LD
LDL
LV
INH
XRT
PNA
HCTZ
DIET
CTMC
PPD
MD
MEALS
PCP
BRIEF
CHEST
MCHC
OF
SSS
FRI
OR
HOURS
CRI
LUNG
PHD
CNS
HEIGHT
TURP
LP
CPAP
SIGNS
SHUFF
FFP
TSH
SGOT
ORIF
D10
ELMVH
T11
PUMC
SC
IR
V6
P1
UPON
CANCER
LENNI
AH
PUOMC
SS
MA
HR
DO
CMC
CAD
USED
GYN
BRBPR
RSV
TGCHO
NTG
SVC
D5W
BYPASS
AVR
QRS
LAD
BB
PO2
PA
AP
JR
BREAST
VA
TWO
FEVER
CPK
HPI
ON
ANEMIA
Q12H
CSF
AM
PE
BIPAP
OP
AORTIC
FIO2
COLOR
SMA
SOB
NT
OT
VAGINA
RLL
PH
IJ
VSS
PRBC
CEA
CTAB
NS
CVP
SL
SBP
G2
NOTE
CAT
JOB
O2
OM
D5
FLUIDS
P0
FAMILY
PAF
L3
BASOS
RRR
COATED
OB
BS
P

In [147]:
CorrectTerms

['TEE',
 'CELL',
 'YEAST',
 'BP',
 'DR',
 'BM',
 'CH',
 'POS',
 'GERD',
 'C1',
 'MARK',
 'SKIN',
 'NEURO',
 'V2',
 'XII',
 'PAIN',
 'US',
 'COR',
 'ACUTE',
 'ATRIAL',
 'RIGHT',
 'CC',
 'ABD',
 'MUMPS',
 'ALT',
 'KETONE',
 'AT',
 'CMV',
 'TOT',
 'SQ',
 'NODES',
 'PO',
 'NON',
 'RATE',
 'FULL',
 'MASS',
 'RETURN',
 'CBC',
 'TR',
 'AR',
 'MED',
 'CA',
 'IN',
 'BID',
 'OBS',
 'WORK',
 'SM',
 'AND',
 'VULVA',
 'SODIUM',
 'STAT',
 'IVC',
 'UTI',
 'ENT',
 'NONE',
 'DP',
 'AV',
 'SITE',
 'NIACIN',
 'WOUND',
 'LD',
 'LV',
 'INH',
 'PPD',
 'MD',
 'PCP',
 'CHEST',
 'OF',
 'FRI',
 'OR',
 'CRI',
 'LUNG',
 'CNS',
 'TURP',
 'LP',
 'SIGNS',
 'ORIF',
 'SC',
 'IR',
 'V6',
 'CANCER',
 'SS',
 'MA',
 'DO',
 'CAD',
 'GYN',
 'RSV',
 'BYPASS',
 'AVR',
 'PO2',
 'PA',
 'AP',
 'JR',
 'BREAST',
 'VA',
 'FEVER',
 'ON',
 'ANEMIA',
 'CSF',
 'AM',
 'PE',
 'OP',
 'AORTIC',
 'COLOR',
 'SMA',
 'SOB',
 'OT',
 'VAGINA',
 'RLL',
 'PH',
 'CEA',
 'NS',
 'SL',
 'SBP',
 'CAT',
 'O2',
 'OM',
 'FAMILY',
 'COATED',
 'OB',
 'BS',
